<a href="https://colab.research.google.com/github/kb-rahul/CourseProject/blob/main/CS598_Deep_Learning_For_Healthcare_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

In [3]:
# read data
df = pd.read_csv("/content/drive/My Drive/DLH Final Project/mimic3/NOTEEVENTS.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.head(2)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...


# Data Preprocessing
Need to focus on 2 tables
1. NOTESEVENTS.csv
2. DIAGNOSES_ICD.csv

Join tables by <subject_id, hadm_id>

Construct 2 datasets from "TEXT" field in NOTESEVENTS.csv for each <subject_id, hadm_id> pair (i.e discharge summary for that admission.)

X1, y and X2, y
x1 = sequence of vectors from word2vec 
x2 = sequence of vectors from tf-idf
y = list of icd codes for <subject_id, hadm_id> i.e. diagnosis maded in ICU admission.

Need to focus on 50 and 100 most commonly diagnosed diseases.

## Common data preprocessing
1. Use NLTK + MetaMap to extract only the symptom related entities (how to use MetaMap is unknown atm.)

2. Filter out sections in discharge summaries that are related to symptoms only, ignore others to speed up things.

3. Negative filters (e.g. "no sign of breath problem").

## Data processing for X1 (Word2Vec)
1. Generate Word2Vec embeddings (currently using Gensim) using "TEXT".

## Data processing for X2 (TF-IDF)
1. Generate TF-IDF vector for each symptom entity.

## Data processing for y
1. multi-hot encoding?
 




In [5]:
import re
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')

eng_stop_words =  stopwords.words('english')

class MySentences(object):
    def __init__(self, dframe):
        self.dframe = dframe
    
    # TODO: Keeping only alpha numeric characters and spaces for now.
    # need to make this better. Find some good libraries.
    def sanitize_text(self, text):
      test = text.strip()
      text = re.sub(r'\s\s+', ' ', text)
      text = re.sub(r'[^a-zA-z0-9\/\.\?\!\s;,\'\-]', '', text)
      text = re.sub(r'[\.\-\/\?\!;,]', ' ', text)
      text = re.sub(r'[\[\]]', '', text)
      return text

    # TODO: adding some basic checks again need to make it better.
    def sanitize_words(self, sentence):
      return [w for w in sentence if w not in eng_stop_words and not w.isdigit()]

    def __iter__(self):
        for idx in range(len(self.dframe)):
          text = self.sanitize_text(self.dframe["TEXT"][idx])
          yield self.sanitize_words(text.split())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df_icd_codes = pd.read_csv("/content/drive/My Drive/DLH Final Project/mimic3/DIAGNOSES_ICD.csv")

In [7]:
df_icd_codes.head(2)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486


In [8]:
counts = df_icd_codes["ICD9_CODE"].value_counts().head(50)
top_icd_codes = counts.index.to_list()
df_admissions_with_top_diseases = df_icd_codes[df_icd_codes["ICD9_CODE"].isin(top_icd_codes)]
df_unique_admissions_with_top_diseases = df_admissions_with_top_diseases[["SUBJECT_ID", "HADM_ID"]].drop_duplicates()

In [9]:
len(df_unique_admissions_with_top_diseases)

55078

In [10]:
filtered_df = pd.merge(df, df_unique_admissions_with_top_diseases, on=["SUBJECT_ID", "HADM_ID"])

In [11]:
filtered_df_disc_sum = filtered_df[filtered_df["CATEGORY"] == 'Discharge summary'].reset_index()
print(len(filtered_df_disc_sum))
print(filtered_df_disc_sum.head(5))

55988
   index  ROW_ID  SUBJECT_ID   HADM_ID   CHARTDATE CHARTTIME STORETIME  \
0      0     174       22532  167853.0  2151-08-04       NaN       NaN   
1      1     170       22532  167853.0  2151-08-04       NaN       NaN   
2     32     175       13702  107527.0  2118-06-14       NaN       NaN   
3     79     176       13702  167118.0  2119-05-25       NaN       NaN   
4    186     177       13702  196489.0  2124-08-18       NaN       NaN   

            CATEGORY DESCRIPTION  CGID  ISERROR  \
0  Discharge summary      Report   NaN      NaN   
1  Discharge summary      Report   NaN      NaN   
2  Discharge summary      Report   NaN      NaN   
3  Discharge summary      Report   NaN      NaN   
4  Discharge summary      Report   NaN      NaN   

                                                TEXT  
0  Admission Date:  [**2151-7-16**]       Dischar...  
1  Admission Date:  [**2151-7-16**]       Dischar...  
2  Admission Date:  [**2118-6-2**]       Discharg...  
3  Admission Date:  [*

Test code for sentence generator

In [12]:
sgen = MySentences(filtered_df_disc_sum)
sentences = list()
for idx, s in enumerate(sgen):
  print(s)
  if idx == 1:
    break

['Admission', 'Date', 'Discharge', 'Date', 'Service', 'ADDENDUM', 'RADIOLOGIC', 'STUDIES', 'Radiologic', 'studies', 'also', 'included', 'chest', 'CT', 'confirmed', 'cavitary', 'lesions', 'left', 'lung', 'apex', 'consistent', 'infectious', 'process', 'tuberculosis', 'This', 'also', 'moderate', 'sized', 'left', 'pleural', 'effusion', 'HEAD', 'CT', 'Head', 'CT', 'showed', 'intracranial', 'hemorrhage', 'mass', 'effect', 'old', 'infarction', 'consistent', 'past', 'medical', 'history', 'ABDOMINAL', 'CT', 'Abdominal', 'CT', 'showed', 'lesions', 'T10', 'sacrum', 'likely', 'secondary', 'osteoporosis', 'These', 'followed', 'repeat', 'imaging', 'outpatient', 'First', 'Name8', 'NamePattern2', 'First', 'Name4', 'NamePattern1', 'Last', 'Name', 'NamePattern1', 'M', 'D', 'MD', 'Number1', 'Dictated', 'ByHospital', 'MEDQUIST36', 'D', 'T', 'JOB', 'Job', 'Number']
['Admission', 'Date', 'Discharge', 'Date', 'HISTORY', 'OF', 'PRESENT', 'ILLNESS', 'The', 'patient', 'year', 'old', 'African', 'American', 'fema

Word2Vec training using gensim.

In [13]:
import gensim
# NOTE: commenting this part so that we dont run this by mistake.
# sgen = MySentences(filtered_df_disc_sum) # a memory-friendly iterator
# model = gensim.models.Word2Vec(sgen, min_count=5, workers=4, sample=1e-05)
# model.save("/content/drive/My Drive/DLH Final Project/mimic3/word2vec.model")

In [14]:
!ls -lh "/content/drive/My Drive/DLH Final Project/mimic3/"

total 3.9G
-rw------- 1 root root  19M Mar 19  2019 DIAGNOSES_ICD.csv
-rw------- 1 root root 3.8G Jun 14  2019 NOTEEVENTS.csv
-rw------- 1 root root  51M Apr 14 07:48 word2vec-4.model
-rw------- 1 root root  78M Apr 13 08:14 word2vec.model


In [15]:
# multi-hot encoding for icd_codes for a given admission

series_admissions_with_top_diseases_grouped = \
df_admissions_with_top_diseases.groupby(['SUBJECT_ID', 
                                         'HADM_ID'])['ICD9_CODE'].apply(list)

df_admissions_with_top_diseases_grouped = \
series_admissions_with_top_diseases_grouped.to_frame().reset_index()

In [16]:
sorted_top_icd_codes = sorted(top_icd_codes)
icd_code_to_idx = dict((k, v) for v, k in enumerate(sorted_top_icd_codes))

In [17]:
# new col to be added to dataframe
multi_hot_ecoding_col = list()
for idx in range(len(df_admissions_with_top_diseases_grouped)):
  icd_codes = df_admissions_with_top_diseases_grouped.iloc[idx]['ICD9_CODE']
  encoding = [0] * 50
  for code in icd_codes:
    encoding[icd_code_to_idx[code]] = 1    
  multi_hot_ecoding_col.append(encoding)

df_admissions_with_top_diseases_grouped['ICD9_CODE_ENCODED'] = multi_hot_ecoding_col

In [18]:
# df_admissions_with_top_diseases_grouped.head(5)
print(len(df_admissions_with_top_diseases_grouped))
print(len(filtered_df_disc_sum))
# TODO: there are more records in filtered_df_disc_sum than 
# df_admissions_with_top_diseases_grouped, why??

55078
55988


In [19]:
filtered_df_disc_sum.head(5)

,index,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,1,170,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
2,32,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
3,79,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
4,186,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...


In [24]:
from keybert import KeyBERT



In [25]:

kw_model = KeyBERT("emilyalsentzer/Bio_ClinicalBERT")
keywords = kw_model.extract_keywords(doc)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/emilyalsentzer_Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
doc = "I had fever from the last few days. I took a crocin and It doesn't seem to reduce"
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), nr_candidates=20)
keywords

[('days took crocin', 0.892),
 ('took crocin doesn', 0.8845),
 ('crocin doesn reduce', 0.8679),
 ('fever days took', 0.799)]

In [26]:
from medacy.model.model import Model

model = Model.load_external('medacy_model_clinical_notes')

[]


In [30]:
annotation = model.predict("The patient was prescribed 1 capsule of Advil for 5 days.")
print(annotation)

[EntTuple(tag='Drug', start=40, end=45, text='Advil'), EntTuple(tag='Duration', start=46, end=56, text='for 5 days')]


In [30]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokebnizer.from_pretrained("samrawal/bert-base-uncased_clinical-ner")

model = AutoModelForTokenClassification.from_pretrained("samrawal/bert-base-uncased_clinical-ner")

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

In [36]:
from transformers import AutoTokenizer, pipeline,  AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("samrawal/bert-base-uncased_clinical-ner")
model = AutoModelForTokenClassification.from_pretrained("samrawal/bert-base-uncased_clinical-ner")
nerpipeline = pipeline('ner', model=model, tokenizer=tokenizer)
text = "I have fever from the last few days"
nerpipeline(text)

[{'end': 12,
  'entity': 'B-problem',
  'index': 3,
  'score': 0.99793506,
  'start': 7,
  'word': 'fever'}]

In [40]:
anno = nerpipeline(filtered_df_disc_sum.TEXT[0])
filtered_df_disc_sum.TEXT[0][175:183]

'cavitary'

In [48]:
def extract_problems(text):
  extractions = nerpipeline(text)
  span_extract = []
  for extract in extractions:
    if 'problem' in extract['entity']:
      span_extract.append((extract['start'], extract['end']))
  
  span_st = span_extract[0][0]
  final_span = []
  final_end = span_extract[0][1]
  for idx, (st, end) in enumerate(span_extract):
    if idx == 0:
      continue
    if st - span_extract[idx-1][1] <= 2:
      final_end = end
      if idx == len(span_extract) - 1:
        final_span.append((span_st, final_end))
    else:
      final_span.append((span_st, final_end))
      span_st = st
      final_end = end

  text_extracts = [text[st:end].replace("\n", " ") for st, end in final_span]
  return text_extracts


extract_problems(filtered_df_disc_sum.TEXT[0])

['cavitary lesions in the left lung apex',
 'infectious process/tuberculosis',
 'moderate-sized left pleural effusion',
 'intracranial hemorrhage',
 'mass effect',
 'old infarction',
 'lesions of T10 and sacrum',
 'osteoporosis']

In [ ]:


df_admissions_with_top_diseases_grouped.head(5)

,SUBJECT_ID,HADM_ID,ICD9_CODE,ICD9_CODE_ENCODED
0,2,163353,"[V3001, V053, V290]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,3,145834,"[0389, 5849, 41071, 4280]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,5,178980,"[V3000, V053, V290]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,6,107064,"[40391, 2767, 2859, V1582]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ..."
4,7,118037,"[V3001, V053, V290]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
# create our final dataset we will be working
# join df_admissions_with_top_diseases_grouped and filtered_df_disc_sum